## Contrato Spot

In [1]:
import pickle

# Load the pickle file containing the model and scaler functions
with open('Spot_Model_and_scaler.pkl', 'rb') as file:
    objects = pickle.load(file)
    
Spot_model = objects['model']
Spot_scaler = objects['scaler']

In [2]:
import numpy as np

# Predict function takes in the features as inputs and return the sales price
def predict(N_material, Lbmo_material, Total_Factura, Costo_Total, margen_venta, model=Spot_model, scaler=Spot_scaler):
    if N_material not in [2020, 1000, 2010, 2000, 2130, 2070, 2210]:
        raise ValueError("Error: The chosen 'Nº Material' is not recognized by the model.")
    
    features = np.array([Lbmo_material, Total_Factura, Costo_Total, margen_venta]).reshape(1,-1)
    scaled = scaler.transform(features)
    # print(scaled)
    inputs = np.hstack([[N_material], scaled[0]])
    input_data = inputs.reshape(1,-1)
    prediction = model.predict(input_data)
    
    return prediction

In [3]:
predict(2020, 32651.334, 270350.28, 263651.55, 6698.73)

f:\Hassan\sales_price\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
f:\Hassan\sales_price\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


array([8.3099])

## Contrato Regular

In [4]:
with open('Regular_models_and_scalers.pkl', 'rb') as file:
    objects = pickle.load(file)
    
Regular_models = objects['models']
Regular_scaler_targets = objects['scaler_target']
Regular_scaler_features = objects['scaler_features']

In [5]:
def predict_sales_price(N_Material, Lbmo_material, Total_Factura, Costo_Total, margen_venta, models=Regular_models, scalers_targets=Regular_scaler_targets, scaler_features=Regular_scaler_features):
    
    if N_Material not in [2020, 2000, 2010]:
        raise ValueError("Error: The chosen 'Nº Material' is not recognized by the model.")
    
    # Scale the features using the fitted scaler
    features_to_scale = np.array([Lbmo_material, Total_Factura, Costo_Total, margen_venta]).reshape(1,-1)
    scaled_features = scaler_features.transform(features_to_scale)
    features = np.hstack([[N_Material], scaled_features[0]]).reshape(1,-1)
    # print(features)
    
    # Predict each component and unscale the predictions
    targets = ['Precio Unitario Venta  US$/Lb', 'Surcharge', 'Factor BQM']
    predictions = {}
    for target in targets:
        pred_scaled = models[target].predict(features)
        pred_unscaled = scalers_targets[target].inverse_transform(pred_scaled.reshape(-1, 1))[0, 0]
        predictions[target] = pred_unscaled
    
    # Calculate the sales price
    sales_price = (predictions['Precio Unitario Venta  US$/Lb'] * predictions['Factor BQM']) + predictions['Surcharge']
    print('Precio:', predictions['Precio Unitario Venta  US$/Lb'])
    print('Factor BQM:', predictions['Factor BQM'])
    print('Surcharge:', predictions['Surcharge'])
    return sales_price

In [6]:
predict_sales_price(2020, 16000.019, 168800, 150869.47, 17930.53)

Precio: 10.1489
Factor BQM: 1.0
Surcharge: 0.06445000001084512


f:\Hassan\sales_price\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
f:\Hassan\sales_price\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
f:\Hassan\sales_price\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
f:\Hassan\sales_price\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


np.float64(10.213350000010845)